In [9]:
import pandas as pd
prices_df = pd.read_excel('Fanduel_Player_Prices.xlsx')

prices_df['Name'] = prices_df['Player'].str.split('(').str[0].str[0:-1]
prices_df['Team'] = prices_df['Player'].apply(lambda st: st[st.find("(")+1:st.find(" - ")])
prices_df['Pos'] = prices_df['Player'].apply(lambda st: st[st.find(" - ")+3:st.find(")")])

prices_df['Team'] = prices_df['Team'].str.replace('JAC','JAX')
prices_df['Name'] = prices_df['Name'].str.replace(' Jr.','')
prices_df['Name'] = prices_df['Name'].str.replace(' Sr.','')
prices_df['Name'] = prices_df['Name'].str.replace(' IV','')
prices_df['Name'] = prices_df['Name'].str.replace(' III','')
prices_df['Name'] = prices_df['Name'].str.replace(' II','')
#prices_df['Name'] = prices_df['Name'].str.replace('A.J.','AJ')
prices_df['Name'] = prices_df['Name'].str.replace('D.J.','DJ')
#prices_df['Name'] = prices_df['Name'].str.replace('K.J.','KJ')
prices_df['Name'] = prices_df['Name'].str.replace('Ken Walker','Kenneth Walker')
prices_df['Key'] = prices_df['Name'] + '-' + prices_df['Team']

prices_df.to_csv('Fanduel_Player_Prices_Clean.csv', index=False)
#prices_df.sort_values(by='Price', ascending=False).head(50)

<ipython-input-9-0d8ec029cf96>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  prices_df['Name'] = prices_df['Name'].str.replace(' Jr.','')
<ipython-input-9-0d8ec029cf96>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  prices_df['Name'] = prices_df['Name'].str.replace(' Sr.','')
<ipython-input-9-0d8ec029cf96>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  prices_df['Name'] = prices_df['Name'].str.replace('D.J.','DJ')


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from datetime import datetime
import numpy as np

In [45]:
week_num = 7

response = requests.get("https://www.rotowire.com/betting/nfl/player-props.php")

if response.status_code != 200:
    print("Error fetching page")
    exit()

soup = BeautifulSoup(response.content, 'html.parser')

obj = soup.find_all('div')
obj2 = str(obj[0])

# initializing substrings
sub1 = "data: [{"
sub2 = "}]"
idx0 = 1
idx1 = 1
idx2 = 1
str2 = ''
str1 = ''

try:
    while 1>0:
        if idx0 == -1 or idx1 == -1 or idx2 == -1:
            #print('complete')
            break
        
        #print('idx0')
        #print(idx0)
        
        idx1 = obj2.find(sub1, idx0)
        #print('idx1') 
        #print(idx1)
        
        idx2 = obj2.find(sub2, idx1)
        #print('idx2') 
        #print(idx2)
        
        str1 = obj2[idx1:idx2+1]

        idx0 = idx2
        idx1 = idx2
        idx2 = idx2+1

        #print(str1)
        #print('')

        str2 = str2 + str1
except:
    pass

str2 = str2.replace('data: [', '')
str2 = str2.strip()
str2

# initializing substrings
sub1 = "gameID"
sub2 = "}"
idx0 = 1
idx1 = 1
idx2 = 1
df2 = pd.DataFrame()

ls = []

while 1>0:
    #print('idx0')
    #print(idx0)

    idx1 = str2.find(sub1, idx0)
    #print('idx1') 
    #print(idx1)

    idx2 = str2.find(sub2, idx1)
    #print('idx2') 
    #print(idx2)
    
    if idx0 == -1 or idx1 == -1 or idx2 == -1:
        print('complete')
        break

    str3 = str2[idx1-2:idx2+1]
    
    json_obj = json.loads(str3)
    df = pd.json_normalize(json_obj)
    
    df2 = pd.concat([df, df2])
    #print(json_obj)
    #print(str3)

    idx0 = idx2
    idx1 = idx2
    idx2 = idx2+1

dt = datetime.now()    
df2['week'] = week_num
df2['runtime'] = dt
#df2.to_csv('ff_test.csv', index=False)
df2.to_csv('ff_test.csv', index=False, mode='a', header=False)

complete


In [46]:
df2 = pd.read_csv('ff_test.csv')
df2 = df2[df2['week']==99]

#max_run_ts = df2['runtime'].max()
#df2 = df2[df2['runtime']==max_run_ts]
#df2 = df2[df2['runtime']==dt]
#print(dt)

over_constant = 110
yds_div = 2.5
recs_div = 25

df2[['draftkings_recs','draftkings_rushrec','draftkings_recyds','draftkings_rushyds','mgm_recs','mgm_rushrec','mgm_recyds','mgm_rushyds','pointsbet_recs','pointsbet_rushrec','pointsbet_recyds','pointsbet_rushyds','fanduel_recs','fanduel_rushrec','fanduel_recyds','fanduel_rushyds','draftkings_comp','draftkings_passatt','draftkings_passyds','fanduel_comp','fanduel_passatt','fanduel_passyds','mgm_comp','mgm_passatt','mgm_passyds','pointsbet_comp','pointsbet_passatt','pointsbet_passyds','mgm_firsttd','mgm_anytd','mgm_lasttd','mgm_twotd','mgm_threetd','draftkings_firsttd','draftkings_anytd','draftkings_lasttd','draftkings_twotd','draftkings_threetd','fanduel_firsttd','fanduel_anytd','fanduel_lasttd','fanduel_twotd','fanduel_threetd','pointsbet_firsttd','pointsbet_anytd','pointsbet_lasttd','pointsbet_twotd','pointsbet_threetd','draftkings_rushydsUnder','draftkings_rushydsOver','fanduel_rushydsUnder','fanduel_rushydsOver','mgm_rushydsUnder','mgm_rushydsOver','pointsbet_rushydsUnder','pointsbet_rushydsOver','draftkings_recydsUnder','draftkings_recydsOver','fanduel_recydsUnder','fanduel_recydsOver','mgm_recydsUnder','mgm_recydsOver','pointsbet_recydsUnder','pointsbet_recydsOver','draftkings_rushrecUnder','draftkings_rushrecOver','mgm_rushrecUnder','mgm_rushrecOver','fanduel_rushrecUnder','fanduel_rushrecOver','pointsbet_rushrecUnder','pointsbet_rushrecOver','draftkings_recsUnder','draftkings_recsOver','mgm_recsUnder','mgm_recsOver','fanduel_recsUnder','fanduel_recsOver','pointsbet_recsUnder','pointsbet_recsOver','fanduel_passydsUnder','fanduel_passydsOver','mgm_passydsUnder','mgm_passydsOver','pointsbet_passydsUnder','pointsbet_passydsOver','draftkings_passydsUnder','draftkings_passydsOver','draftkings_passattUnder','draftkings_passattOver','fanduel_passattUnder','fanduel_passattOver','mgm_passattUnder','mgm_passattOver','pointsbet_passattUnder','pointsbet_passattOver','fanduel_compUnder','fanduel_compOver','mgm_compUnder','mgm_compOver','draftkings_compUnder','draftkings_compOver','pointsbet_compUnder','pointsbet_compOver','draftkings_threetdUnder','draftkings_threetdOver','fanduel_threetdUnder','fanduel_threetdOver','mgm_threetdUnder','mgm_threetdOver','pointsbet_threetdUnder','pointsbet_threetdOver','draftkings_twotdUnder','draftkings_twotdOver','pointsbet_twotdUnder','pointsbet_twotdOver','fanduel_twotdUnder','fanduel_twotdOver','mgm_twotdUnder','mgm_twotdOver','mgm_lasttdUnder','mgm_lasttdOver','draftkings_lasttdUnder','draftkings_lasttdOver','fanduel_lasttdUnder','fanduel_lasttdOver','pointsbet_lasttdUnder','pointsbet_lasttdOver','mgm_anytdUnder','mgm_anytdOver','draftkings_anytdUnder','draftkings_anytdOver','fanduel_anytdUnder','fanduel_anytdOver','pointsbet_anytdUnder','pointsbet_anytdOver','mgm_firsttdUnder','mgm_firsttdOver','draftkings_firsttdUnder','draftkings_firsttdOver','fanduel_firsttdUnder','fanduel_firsttdOver','pointsbet_firsttdUnder','pointsbet_firsttdOver']] = df2[['draftkings_recs','draftkings_rushrec','draftkings_recyds','draftkings_rushyds','mgm_recs','mgm_rushrec','mgm_recyds','mgm_rushyds','pointsbet_recs','pointsbet_rushrec','pointsbet_recyds','pointsbet_rushyds','fanduel_recs','fanduel_rushrec','fanduel_recyds','fanduel_rushyds','draftkings_comp','draftkings_passatt','draftkings_passyds','fanduel_comp','fanduel_passatt','fanduel_passyds','mgm_comp','mgm_passatt','mgm_passyds','pointsbet_comp','pointsbet_passatt','pointsbet_passyds','mgm_firsttd','mgm_anytd','mgm_lasttd','mgm_twotd','mgm_threetd','draftkings_firsttd','draftkings_anytd','draftkings_lasttd','draftkings_twotd','draftkings_threetd','fanduel_firsttd','fanduel_anytd','fanduel_lasttd','fanduel_twotd','fanduel_threetd','pointsbet_firsttd','pointsbet_anytd','pointsbet_lasttd','pointsbet_twotd','pointsbet_threetd','draftkings_rushydsUnder','draftkings_rushydsOver','fanduel_rushydsUnder','fanduel_rushydsOver','mgm_rushydsUnder','mgm_rushydsOver','pointsbet_rushydsUnder','pointsbet_rushydsOver','draftkings_recydsUnder','draftkings_recydsOver','fanduel_recydsUnder','fanduel_recydsOver','mgm_recydsUnder','mgm_recydsOver','pointsbet_recydsUnder','pointsbet_recydsOver','draftkings_rushrecUnder','draftkings_rushrecOver','mgm_rushrecUnder','mgm_rushrecOver','fanduel_rushrecUnder','fanduel_rushrecOver','pointsbet_rushrecUnder','pointsbet_rushrecOver','draftkings_recsUnder','draftkings_recsOver','mgm_recsUnder','mgm_recsOver','fanduel_recsUnder','fanduel_recsOver','pointsbet_recsUnder','pointsbet_recsOver','fanduel_passydsUnder','fanduel_passydsOver','mgm_passydsUnder','mgm_passydsOver','pointsbet_passydsUnder','pointsbet_passydsOver','draftkings_passydsUnder','draftkings_passydsOver','draftkings_passattUnder','draftkings_passattOver','fanduel_passattUnder','fanduel_passattOver','mgm_passattUnder','mgm_passattOver','pointsbet_passattUnder','pointsbet_passattOver','fanduel_compUnder','fanduel_compOver','mgm_compUnder','mgm_compOver','draftkings_compUnder','draftkings_compOver','pointsbet_compUnder','pointsbet_compOver','draftkings_threetdUnder','draftkings_threetdOver','fanduel_threetdUnder','fanduel_threetdOver','mgm_threetdUnder','mgm_threetdOver','pointsbet_threetdUnder','pointsbet_threetdOver','draftkings_twotdUnder','draftkings_twotdOver','pointsbet_twotdUnder','pointsbet_twotdOver','fanduel_twotdUnder','fanduel_twotdOver','mgm_twotdUnder','mgm_twotdOver','mgm_lasttdUnder','mgm_lasttdOver','draftkings_lasttdUnder','draftkings_lasttdOver','fanduel_lasttdUnder','fanduel_lasttdOver','pointsbet_lasttdUnder','pointsbet_lasttdOver','mgm_anytdUnder','mgm_anytdOver','draftkings_anytdUnder','draftkings_anytdOver','fanduel_anytdUnder','fanduel_anytdOver','pointsbet_anytdUnder','pointsbet_anytdOver','mgm_firsttdUnder','mgm_firsttdOver','draftkings_firsttdUnder','draftkings_firsttdOver','fanduel_firsttdUnder','fanduel_firsttdOver','pointsbet_firsttdUnder','pointsbet_firsttdOver']].apply(pd.to_numeric)
df2 = df2[['gameID','playerID','name','team','opp','draftkings_recyds','draftkings_recydsOver','mgm_recyds','mgm_recydsOver','pointsbet_recyds','pointsbet_recydsOver','fanduel_recyds','fanduel_recydsOver','draftkings_anytd','draftkings_anytdOver','mgm_anytd','mgm_anytdOver','pointsbet_anytd','pointsbet_anytdOver','fanduel_anytd','fanduel_anytdOver','draftkings_rushyds','draftkings_rushydsOver','mgm_rushyds','mgm_rushydsOver','pointsbet_rushyds','pointsbet_rushydsOver','fanduel_rushyds','fanduel_rushydsOver','draftkings_passyds','draftkings_passydsOver','mgm_passyds','mgm_passydsOver','pointsbet_passyds','pointsbet_passydsOver','fanduel_passyds','fanduel_passydsOver','draftkings_recs','draftkings_recsOver','mgm_recs','mgm_recsOver','pointsbet_recs','pointsbet_recsOver','fanduel_recs','fanduel_recsOver','runtime']]

df2['draftkings_recyds'] = np.where(df2['draftkings_recydsOver']<0,df2['draftkings_recyds']-(df2['draftkings_recydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['draftkings_recyds']-(df2['draftkings_recydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['mgm_recyds'] = np.where(df2['mgm_recydsOver']<0,df2['mgm_recyds']-(df2['mgm_recydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['mgm_recyds']-(df2['mgm_recydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['pointsbet_recyds'] = np.where(df2['pointsbet_recydsOver']<0,df2['pointsbet_recyds']-(df2['pointsbet_recydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['pointsbet_recyds']-(df2['pointsbet_recydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['fanduel_recyds'] = np.where(df2['fanduel_recydsOver']<0,df2['fanduel_recyds']-(df2['fanduel_recydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['fanduel_recyds']-(df2['fanduel_recydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)

df2['draftkings_rushyds'] = np.where(df2['draftkings_rushydsOver']<0,df2['draftkings_rushyds']-(df2['draftkings_rushydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['draftkings_rushyds']-(df2['draftkings_rushydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['mgm_rushyds'] = np.where(df2['mgm_rushydsOver']<0,df2['mgm_rushyds']-(df2['mgm_rushydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['mgm_rushyds']-(df2['mgm_rushydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['pointsbet_rushyds'] = np.where(df2['pointsbet_rushydsOver']<0,df2['pointsbet_rushyds']-(df2['pointsbet_rushydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['pointsbet_rushyds']-(df2['pointsbet_rushydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)
df2['fanduel_rushyds'] = np.where(df2['fanduel_rushydsOver']<0,df2['fanduel_rushyds']-(df2['fanduel_rushydsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/yds_div,df2['fanduel_rushyds']-(df2['fanduel_rushydsOver'].mask(pd.isnull, over_constant)-over_constant)/yds_div)

df2['draftkings_passyds'] = np.where(df2['draftkings_passydsOver']<0,df2['draftkings_passyds']-(df2['draftkings_passydsOver'].mask(pd.isnull, -1*over_constant)+over_constant),df2['draftkings_passyds']-(df2['draftkings_passydsOver'].mask(pd.isnull, over_constant)-over_constant))
df2['mgm_passyds'] = np.where(df2['mgm_passydsOver']<0,df2['mgm_passyds']-(df2['mgm_passydsOver'].mask(pd.isnull, -1*over_constant)+over_constant),df2['mgm_passyds']-(df2['mgm_passydsOver'].mask(pd.isnull, over_constant)-over_constant))
df2['pointsbet_passyds'] = np.where(df2['pointsbet_passydsOver']<0,df2['pointsbet_passyds']-(df2['pointsbet_passydsOver'].mask(pd.isnull, -1*over_constant)+over_constant),df2['pointsbet_passyds']-(df2['pointsbet_passydsOver'].mask(pd.isnull, over_constant)-over_constant))
df2['fanduel_passyds'] = np.where(df2['fanduel_passydsOver']<0,df2['fanduel_passyds']-(df2['fanduel_passydsOver'].mask(pd.isnull, -1*over_constant)+over_constant),df2['fanduel_passyds']-(df2['fanduel_passydsOver'].mask(pd.isnull, over_constant)-over_constant))

df2['draftkings_recs'] = np.where(df2['draftkings_recsOver']<0,df2['draftkings_recs']-(df2['draftkings_recsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/recs_div,df2['draftkings_recs']-(df2['draftkings_recsOver'].mask(pd.isnull, over_constant)-over_constant)/recs_div)
df2['mgm_recs'] = np.where(df2['mgm_recsOver']<0,df2['mgm_recs']-(df2['mgm_recsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/recs_div,df2['mgm_recs']-(df2['mgm_recsOver'].mask(pd.isnull, over_constant)-over_constant)/recs_div)
df2['pointsbet_recs'] = np.where(df2['pointsbet_recsOver']<0,df2['pointsbet_recs']-(df2['pointsbet_recsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/recs_div,df2['pointsbet_recs']-(df2['pointsbet_recsOver'].mask(pd.isnull, over_constant)-over_constant)/recs_div)
df2['fanduel_recs'] = np.where(df2['fanduel_recsOver']<0,df2['fanduel_recs']-(df2['fanduel_recsOver'].mask(pd.isnull, -1*over_constant)+over_constant)/recs_div,df2['fanduel_recs']-(df2['fanduel_recsOver'].mask(pd.isnull, over_constant)-over_constant)/recs_div)

#df2
#df2.sort_values(by='draftkings_passydsOver', ascending = True).head(5)
#df2.to_csv('test.csv')
df2 = df2[['gameID','playerID','name','team','opp','draftkings_recyds','mgm_recyds','pointsbet_recyds','fanduel_recyds','draftkings_anytd','mgm_anytd','pointsbet_anytd','fanduel_anytd','draftkings_rushyds','mgm_rushyds','pointsbet_rushyds','fanduel_rushyds','draftkings_passyds','mgm_passyds','pointsbet_passyds','fanduel_passyds','draftkings_recs', 'mgm_recs', 'pointsbet_recs', 'fanduel_recs', 'runtime']]
df3 = df2.groupby(['gameID','playerID','name','team','opp','runtime']).max().reset_index()

#Optimistic
#df3['recyds'] = df3[['draftkings_recyds', 'mgm_recyds', 'pointsbet_recyds', 'fanduel_recyds']].max(axis=1)
#df3['anytd'] = df3[['draftkings_anytd', 'mgm_anytd', 'pointsbet_anytd', 'fanduel_anytd']].min(axis=1)
#df3['rushyds'] = df3[['draftkings_rushyds', 'mgm_rushyds', 'pointsbet_rushyds', 'fanduel_rushyds']].max(axis=1)
#df3['passyds'] = df3[['draftkings_passyds', 'mgm_passyds', 'pointsbet_passyds', 'fanduel_passyds']].max(axis=1)
#df3['recs'] = df3[['draftkings_recs', 'mgm_recs', 'pointsbet_recs', 'fanduel_recs']].max(axis=1)

#Safe
df3['recyds'] = df3[['draftkings_recyds', 'mgm_recyds', 'pointsbet_recyds', 'fanduel_recyds']].min(axis=1)
df3['anytd'] = df3[['draftkings_anytd', 'mgm_anytd', 'pointsbet_anytd', 'fanduel_anytd']].max(axis=1)
df3['rushyds'] = df3[['draftkings_rushyds', 'mgm_rushyds', 'pointsbet_rushyds', 'fanduel_rushyds']].min(axis=1)
df3['passyds'] = df3[['draftkings_passyds', 'mgm_passyds', 'pointsbet_passyds', 'fanduel_passyds']].min(axis=1)
df3['recs'] = df3[['draftkings_recs', 'mgm_recs', 'pointsbet_recs', 'fanduel_recs']].min(axis=1)

#df4 = df3[['gameID','playerID','name','team','opp','recyds','anytd','rushyds','passyds']]
df4 = df3[['gameID','playerID','name','team','opp','recyds','anytd','rushyds','passyds','recs','runtime']]
df4['recyds'] = df4['recyds'].fillna(0)
df4['anytd'] = df4['anytd'].fillna(5000)
df4['rushyds'] = df4['rushyds'].fillna(0)
df4['passyds'] = df4['passyds'].fillna(0)
df4['recs'] = df4['recs'].fillna(0)

df4['anytd_pct'] = np.where(df4['anytd']<0,abs(df4['anytd'])/(abs(df4['anytd'])+100), np.where(df4['anytd']>0,100/(df4['anytd']+100),0))
#df4['anytd_pct'] = np.where(df4['anytd_pct']>0.5,1/df4['anytd_pct'],df4['anytd_pct'])

#df4['anytd_pct'] = df4['anytd'].rank(pct = True, method='min')
#df4['anytd_pct'] = 1-df4['anytd_pct']

df4['recyds_fpts'] = (df4['recyds']/10)
df4['td_fpts'] = (df4['anytd_pct']*6)
df4['rush_fpts'] = (df4['rushyds']/10)
df4['passyds_fpts'] = (df4['passyds']/25*1.5)
df4['recs_fpts'] = df4['recs']*0.5

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#df4[['recyds_scaler','anytd_scaler','rushyds_scaler','passyds_scaler']] = scaler.fit_transform(df4[['recyds','anytd','rushyds','passyds']])
#df4['anytd_scaler'] = 1 - df4['anytd_scaler']

#df4['points'] = df4['recyds_fpts'] + df4['td_fpts'] + df4['rush_fpts'] + df4['passyds_fpts']
df4['points'] = df4['recyds_fpts'] + df4['td_fpts'] + df4['rush_fpts'] + df4['passyds_fpts'] + df4['recs_fpts']
#df4.sort_values(by='proj_pts2', ascending=False).head(50)
df4.to_csv('ff_test2.csv')

<ipython-input-46-0d90dd513801>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['recyds'] = df4['recyds'].fillna(0)
<ipython-input-46-0d90dd513801>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['anytd'] = df4['anytd'].fillna(5000)
<ipython-input-46-0d90dd513801>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [5]:
from itertools import permutations
from pulp import *
import pandas as pd

prices_df = pd.read_csv('Fanduel_Player_Prices_Clean.csv')
df = pd.read_csv('ff_test2.csv')

df['Key'] = df['name'] + '-' + df['team']
df = pd.merge(df, prices_df, on='Key', how='left')
df['Value'] = np.where(df['Price']>0,df['points']/df['Price']*1000,0)
df.to_csv('ff_test3.csv',index=False)

In [33]:
df = pd.read_csv('ff_test3.csv') 
df = df[df['Price']>0]

n = len(pd.unique(df['playerID']))
print(str(n) + ' players')

#df['points'] = df['points']-df['td_fpts']

#ls = []
df = df[~df['name'].isin(ls)]
df = df[~df['Team'].isin(['PHI','DAL','DEN','LAC'])]
#df['cap'] = np.where(df['Price'].between(4000, 5900), 1, 0)

SALARY_CAP = 60000-4300

salaries = {}
points = {}
#caps = {}

for pos in df.Pos.unique():
    available_pos = df[df.Pos == pos]
    salary = list(available_pos[["name","Price"]].set_index("name").to_dict().values())[0]
    point = list(available_pos[["name","points"]].set_index("name").to_dict().values())[0]
    cap = list(available_pos[["name","cap"]].set_index("name").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point
    #caps[pos] = cap
    
pos_num_available_max = {
    "QB": 1,
    "RB": 3,
    "WR": 4,
    "TE": 2
}

pos_num_available_min = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 1
}

total_players = 8

_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
sal_constraints = []

# Setting up the reward
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) >= pos_num_available_min[k]
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available_max[k]
    #sal_constraints += lpSum([caps[k][i] * _vars[k][i] for i in v])

#prob += lpSum(sal_constraints) >= 4
prob += lpSum(_vars) == total_players
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

prob.solve()

def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

summary(prob)

250 players
Variables:

QB_Kyler_Murray = 1.0
RB_Aaron_Jones = 1.0
RB_Christian_McCaffrey = 1.0
RB_Deon_Jackson = 1.0
TE_Hayden_Hurst = 1.0
WR_Brandon_Aiyuk = 1.0
WR_Marquez_Valdes_Scantling = 1.0
WR_Stefon_Diggs = 1.0
---------------------------------------

Constraints:
8300.0*1.0 + 7300.0*1.0 + 8700.0*1.0 + 5600.0*1.0 + 4900.0*1.0 + 5800.0*1.0 + 5800.0*1.0 + 8900.0*1.0 = 55300.0
---------------------------------------

Score:
20.74*1.0 + 13.359302325581394*1.0 + 15.536666666666664*1.0 + 9.257692307692308*1.0 + 6.963478260869565*1.0 + 9.06*1.0 + 8.746666666666666*1.0 + 15.46*1.0 = 99.12380622747659


In [32]:
for v in prob.variables():
    if v.varValue != 0:
        plyr_nm = v.name
        plyr_nm = plyr_nm[3:]
        plyr_nm = plyr_nm.replace('_', ' ')
        ls.append(plyr_nm)
ls

['Josh Allen',
 'Darrell Henderson',
 'Eno Benjamin',
 'Rhamondre Stevenson',
 'George Kittle',
 'Adam Thielen',
 'Cooper Kupp',
 'Rondale Moore',
 'Patrick Mahomes',
 'AJ Dillon',
 'Devin Singletary',
 'Saquon Barkley',
 'Mark Andrews',
 'DJ Moore',
 'Deebo Samuel',
 'Romeo Doubs']

In [31]:
df = pd.read_csv('ff_test2.csv')
runtime_ls = list(df.sort_values(by='runtime', ascending=False)['runtime'].unique())
df_latest = df[df['runtime']==runtime_ls[0]]
df_earlier = df[df['runtime']==runtime_ls[1]]     

df_merge = pd.merge(df_latest, df_earlier, on=['gameID','playerID'], how='inner')
df_merge['Diff']=df_merge['points_x']-df_merge['points_y']

df_merge['recyds_x_bool'] = df_merge['recyds_x']>0
df_merge['recyds_x_bool'] = df_merge['recyds_x_bool'].replace({True: 1, False: -1})
df_merge['recyds_y_bool'] = df_merge['recyds_y']>0
df_merge['recyds_y_bool'] = df_merge['recyds_y_bool'].replace({True: 1, False: -1})
df_merge['recyds_bool'] = df_merge['recyds_x_bool']*df_merge['recyds_y_bool']

df_merge['rushyds_x_bool'] = df_merge['rushyds_x']>0
df_merge['rushyds_x_bool'] = df_merge['rushyds_x_bool'].replace({True: 1, False: -1})
df_merge['rushyds_y_bool'] = df_merge['rushyds_y']>0
df_merge['rushyds_y_bool'] = df_merge['rushyds_y_bool'].replace({True: 1, False: -1})
df_merge['rushyds_bool'] = df_merge['rushyds_x_bool']*df_merge['rushyds_y_bool']

df_merge['passyds_x_bool'] = df_merge['passyds_x']>0
df_merge['passyds_x_bool'] = df_merge['passyds_x_bool'].replace({True: 1, False: -1})
df_merge['passyds_y_bool'] = df_merge['passyds_y']>0
df_merge['passyds_y_bool'] = df_merge['passyds_y_bool'].replace({True: 1, False: -1})
df_merge['passyds_bool'] = df_merge['passyds_x_bool']*df_merge['passyds_y_bool']

df_merge['recs_x_bool'] = df_merge['recs_x']>0
df_merge['recs_x_bool'] = df_merge['recs_x_bool'].replace({True: 1, False: -1})
df_merge['recs_y_bool'] = df_merge['recs_y']>0
df_merge['recs_y_bool'] = df_merge['recs_y_bool'].replace({True: 1, False: -1})
df_merge['recs_bool'] = df_merge['recs_x_bool']*df_merge['recs_y_bool']

df_merge['anytd_x_bool'] = df_merge['anytd_x']>=5000
df_merge['anytd_x_bool'] = df_merge['anytd_x_bool'].replace({True: 1, False: -1})
df_merge['anytd_y_bool'] = df_merge['anytd_y']>=5000
df_merge['anytd_y_bool'] = df_merge['anytd_y_bool'].replace({True: 1, False: -1})
df_merge['anytd_bool'] = df_merge['anytd_x_bool']*df_merge['anytd_y_bool']

#df_merge = df_merge[np.where((df_merge['anytd_bool']==1)&(df_merge['recs_bool']==1)&(df_merge['recyds_bool']==1)&(df_merge['rushyds_bool']==1)&(df_merge['passyds_bool']==1))]
df_merge['bool'] = df_merge['anytd_bool']+df_merge['recs_bool']+df_merge['recyds_bool']+df_merge['rushyds_bool']+df_merge['passyds_bool']
df_merge = df_merge[df_merge['bool'] == 5]

df_merge.sort_values(by='Diff', ascending=False).head(50)

IndexError: list index out of range

In [20]:
df_merge.to_csv('Diff.csv')